# 01. Importing libraries and data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# project folder path
path = r'/Users/achestnut/Desktop/Instacart Data Analysis'

In [3]:
# import customer data set
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [5]:
# import orders_products_merged3
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged3.pkl'))

# 02. Wrangle customer data

In [6]:
# check shape
df_cust.shape

(206209, 10)

In [7]:
# check head
df_cust.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [9]:
# rename 'Surnam' column
df_cust.rename(columns = {'Surnam': 'Last Name'}, inplace = True)

In [12]:
# rename 'n_dependents' column
df_cust.rename(columns = {'n_dependants': 'Number_of_dependants'}, inplace = True)

In [13]:
# check output
df_cust.head()

user_id First Name Last Name  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   Number_of_dependants fam_status  income  
0                     3    married  165665  
1                     0     single   59285  
2                     2    married   99568  
3                     0     single   42049  
4                     1    married   40374

# 03. Fundamental data quality and consistency checks

In [14]:
# data types
df_cust.dtypes

user_id                  int64
First Name              object
Last Name               object
Gender                  object
STATE                   object
Age                      int64
date_joined             object
Number_of_dependants     int64
fam_status              object
income                   int64
dtype: object

In [15]:
# change 'user_id' to a string as the id number represents a category
df_cust['user_id'] = df_cust['user_id'].astype('str')

In [16]:
# data types after change
df_cust.dtypes

user_id                 object
First Name              object
Last Name               object
Gender                  object
STATE                   object
Age                      int64
date_joined             object
Number_of_dependants     int64
fam_status              object
income                   int64
dtype: object

In [17]:
# missing values
df_cust.isnull().sum()

user_id                     0
First Name              11259
Last Name                   0
Gender                      0
STATE                       0
Age                         0
date_joined                 0
Number_of_dependants        0
fam_status                  0
income                      0
dtype: int64

In [18]:
# 'First Name' missing values to nan
df_nan = df_cust[df_cust['First Name'].isnull() == True]

In [19]:
df_nan

user_id First Name Last Name  Gender         STATE  Age date_joined  \
53       76659        NaN   Gilbert    Male      Colorado   26    1/1/2017   
73       13738        NaN     Frost  Female     Louisiana   39    1/1/2017   
82       89996        NaN    Dawson  Female        Oregon   52    1/1/2017   
99       96166        NaN   Oconnor    Male      Oklahoma   51    1/1/2017   
105      29778        NaN    Dawson  Female          Utah   63    1/1/2017   
...        ...        ...       ...     ...           ...  ...         ...   
206038  121317        NaN    Melton    Male  Pennsylvania   28   3/31/2020   
206044  200799        NaN  Copeland  Female        Hawaii   52    4/1/2020   
206090  167394        NaN     Frost  Female        Hawaii   61    4/1/2020   
206162  187532        NaN     Floyd  Female    California   39    4/1/2020   
206171  116898        NaN   Delgado  Female      Colorado   23    4/1/2020   

        Number_of_dependants fam_status  income  
53                         2    married   41709  
73                         0     single   82518  
82                         3    married  117099  
99                         1    married  155673  
105                        3    married  151819  
...                      ...        ...     ...  
206038                     3    married   87783  
206044                     2    married  108488  
206090                     1    married   45275  
206162                     0     single   56325  
206171                     2    married   59222  

[11259 rows x 10 columns]

In [20]:
# mixed type data
for col in df_cust.columns.tolist():
    weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis =1)
    if len (df_cust[weird]) > 0:
        print(col)

First Name


In [21]:
# 'First Name' to string after missing values changed to nan
df_cust['First Name'] = df_cust['First Name'].astype('str')

In [22]:
# data types after change 2
df_cust.dtypes

user_id                 object
First Name              object
Last Name               object
Gender                  object
STATE                   object
Age                      int64
date_joined             object
Number_of_dependants     int64
fam_status              object
income                   int64
dtype: object

In [23]:
# missing values 2
df_cust.isnull().sum()

user_id                 0
First Name              0
Last Name               0
Gender                  0
STATE                   0
Age                     0
date_joined             0
Number_of_dependants    0
fam_status              0
income                  0
dtype: int64

In [24]:
# duplicates
df_duplicates = df_cust[df_cust.duplicated()]

In [25]:
df_duplicates

Empty DataFrame
Columns: [user_id, First Name, Last Name, Gender, STATE, Age, date_joined, Number_of_dependants, fam_status, income]
Index: []

# 04. Combine data sets

In [26]:
# check shape customer data
df_cust.shape

(206209, 10)

In [27]:
# check shape ords_prods_merge data
ords_prods_merge.shape

(32404859, 25)

In [28]:
# data types customer data
df_cust.dtypes

user_id                 object
First Name              object
Last Name               object
Gender                  object
STATE                   object
Age                      int64
date_joined             object
Number_of_dependants     int64
fam_status              object
income                   int64
dtype: object

In [29]:
# data types ords_prods_merge data
ords_prods_merge.dtypes

order_id                     int64
user_id                      int64
eval_set                    object
order_number                 int64
orders_day_of_week           int64
hour_of_order                int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
busiest_day                 object
price_range_loc             object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
mean_price                 float64
spending_flag               object
median_days                float64
frequency_flag              object
dtype: object

In [30]:
# 'user_id' to string in ords_prods_merge to be same as 'user_id' in customer data
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')

In [31]:
# data types ords_prods_merge data
ords_prods_merge.dtypes

order_id                     int64
user_id                     object
eval_set                    object
order_number                 int64
orders_day_of_week           int64
hour_of_order                int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
busiest_day                 object
price_range_loc             object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
mean_price                 float64
spending_flag               object
median_days                float64
frequency_flag              object
dtype: object

In [32]:
# merge 2 data sets via 'user_id'
df_combined = ords_prods_merge.merge(df_cust, on = 'user_id')

In [33]:
df_combined

order_id user_id eval_set  order_number  orders_day_of_week  \
0          2539329       1    prior             1                   2   
1          2398795       1    prior             2                   3   
2           473747       1    prior             3                   3   
3          2254736       1    prior             4                   4   
4           431534       1    prior             5                   4   
...            ...     ...      ...           ...                 ...   
32404854    156685  106143    prior            26                   4   
32404855    484769   66343    prior             1                   6   
32404856   1561557   66343    prior             2                   1   
32404857    276317   66343    prior             3                   6   
32404858   2922475   66343    prior             4                   1   

          hour_of_order  days_since_prior_order  product_id  \
0                     8                     0.0         196   
1                     7                    15.0         196   
2                    12                    21.0         196   
3                     7                    29.0         196   
4                    15                    28.0         196   
...                 ...                     ...         ...   
32404854             23                     5.0       19675   
32404855             11                     0.0       47210   
32404856             11                    30.0       47210   
32404857             15                    19.0       47210   
32404858             12                    30.0       47210   

          add_to_cart_order  reordered  ...         frequency_flag  \
0                         1          0  ...       Regular customer   
1                         1          1  ...       Regular customer   
2                         1          1  ...       Regular customer   
3                         1          1  ...       Regular customer   
4                         1          1  ...       Regular customer   
...                     ...        ...  ...                    ...   
32404854                  1          1  ...      Frequent customer   
32404855                  1          0  ...  Non-frequent customer   
32404856                  1          1  ...  Non-frequent customer   
32404857                  1          1  ...  Non-frequent customer   
32404858                  1          1  ...  Non-frequent customer   

          First Name  Last Name  Gender      STATE Age date_joined  \
0              Linda     Nguyen  Female    Alabama  31   2/17/2019   
1              Linda     Nguyen  Female    Alabama  31   2/17/2019   
2              Linda     Nguyen  Female    Alabama  31   2/17/2019   
3              Linda     Nguyen  Female    Alabama  31   2/17/2019   
4              Linda     Nguyen  Female    Alabama  31   2/17/2019   
...              ...        ...     ...        ...  ..         ...   
32404854      Gerald      Yates    Male     Hawaii  25   5/26/2017   
32404855  Jacqueline     Arroyo  Female  Tennessee  22   9/12/2017   
32404856  Jacqueline     Arroyo  Female  Tennessee  22   9/12/2017   
32404857  Jacqueline     Arroyo  Female  Tennessee  22   9/12/2017   
32404858  Jacqueline     Arroyo  Female  Tennessee  22   9/12/2017   

         Number_of_dependants fam_status  income  
0                           3    married   40423  
1                           3    married   40423  
2                           3    married   40423  
3                           3    married   40423  
4                           3    married   40423  
...                       ...        ...     ...  
32404854                    0     single   53755  
32404855                    3    married   46151  
32404856                    3    married   46151  
32404857                    3    married   46151  
32404858                    3    married   46151  

[32404859 rows x 34 columns]

In [34]:
# change case of column names to be consistent throughout 
df_combined.rename(columns = {'First Name': 'first_name', 'Last Name' : 'last_name', 'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'Number_of_dependants' : 'number_of_dependants', 'fam_status' : 'marital_status'}, inplace = True)

In [35]:
df_combined.head()

order_id user_id eval_set  order_number  orders_day_of_week  hour_of_order  \
0   2539329       1    prior             1                   2              8   
1   2398795       1    prior             2                   3              7   
2    473747       1    prior             3                   3             12   
3   2254736       1    prior             4                   4              7   
4    431534       1    prior             5                   4             15   

   days_since_prior_order  product_id  add_to_cart_order  reordered  ...  \
0                     0.0         196                  1          0  ...   
1                    15.0         196                  1          1  ...   
2                    21.0         196                  1          1  ...   
3                    29.0         196                  1          1  ...   
4                    28.0         196                  1          1  ...   

     frequency_flag  first_name  last_name  gender    state age date_joined  \
0  Regular customer       Linda     Nguyen  Female  Alabama  31   2/17/2019   
1  Regular customer       Linda     Nguyen  Female  Alabama  31   2/17/2019   
2  Regular customer       Linda     Nguyen  Female  Alabama  31   2/17/2019   
3  Regular customer       Linda     Nguyen  Female  Alabama  31   2/17/2019   
4  Regular customer       Linda     Nguyen  Female  Alabama  31   2/17/2019   

  number_of_dependants marital_status  income  
0                    3        married   40423  
1                    3        married   40423  
2                    3        married   40423  
3                    3        married   40423  
4                    3        married   40423  

[5 rows x 34 columns]

# 05. Export as pickle

In [36]:
df_combined.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'cust_ords_prods_merge.pkl'))